In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sys
sys.path.append('./Network_Graphs/')

# LeNet-300-100 Model

In [19]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Xtr = train_images.reshape(train_images.shape[0],-1)/255
Xte = test_images.reshape(test_images.shape[0],-1)/255
ytr = train_labels
yte = test_labels

[[W1,b1,W2,b2,W3,b3],inp_mask] = np.load('./Saved_Models/LeNet300_model.npy')
Xte_masked = Xte[:,inp_mask]
a1 = np.maximum(Xte_masked.dot(W1) + b1, 0)
a2 = np.maximum(a1.dot(W2) + b2, 0)
z3 = a2.dot(W3) + b3
test_acc = accuracy_score(np.argmax(z3,axis=1),yte)

print(Xte_masked.shape)
print(a1.shape)
print(a2.shape)
print(round(test_acc * 100, 2))

(10000, 316)
(10000, 66)
(10000, 25)
98.42


# LeNet5 Model

In [18]:
# Load Dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Xtr = train_images.reshape(train_images.shape[0],-1)/255
Xte = test_images.reshape(test_images.shape[0],-1)/255
ytr = train_labels
yte = test_labels

# Load Model
from LeNet5_MNIST import LeNet_5
network,fla_sel = np.load('./Saved_Models/LeNet5_model.npy')
mLeNet5 = LeNet_5(network,fla_sel)
test_layer,test_logits = mLeNet5.inference(Xte.astype(np.float32))

# Accuracy Test
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
m_result = sess.run(test_logits)
test_acc = accuracy_score(np.argmax(m_result,axis=1),yte)

print(mLeNet5.W_conv1.shape)
print(mLeNet5.W_conv2.shape)
print(mLeNet5.W_fc1.shape)
print(mLeNet5.W_fc2.shape)
print(round(test_acc * 100,2))

/u/yl16/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(5, 5, 1, 4)
(5, 5, 4, 11)
(144, 24)
(24, 10)
99.33


# VGG16 Model

In [24]:
# CIFAR-10 Data
CIFAR10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = CIFAR10.load_data()
mean = 120.707
std = 64.15
Xtr = (train_images - mean)/(std + 1e-7)
Xte = (test_images - mean)/(std + 1e-7) # The way for data standardization a bit different from MNIST
ytr = train_labels
yte = test_labels

import VGG16_CIFAR10
tf.reset_default_graph()
m_param = np.load('./Saved_Models/VGG16_model.npy')
mVGG16 = VGG16_CIFAR10.VGG16(m_param)
mVGG16.print_shape()

dummy, test_logits = mVGG16.inference(Xte.astype(np.float32),False,1)
correct_prediction = tf.equal(tf.argmax(test_logits, 1), yte.reshape(-1))
acc_score = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

# Add the variable initializer Op.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

test_acc = sess.run(acc_score)
print(round(test_acc*100,2))

(Input)3 - [44, 52, 70, 86, 157, 95, 59, 118, 58, 58, 58, 81, 81, 81] - 10(Output)
93.61
